In [1]:
import funzioni_progetto_webdata as fpw

In [2]:
#se non abbiamo un modello salvato lo creaiamo allenandolo
model=fpw.ModelEmotionDetection("text_emotion_esercizio.csv")

In [5]:
#se abbiamo un modello salvato lo carichiamo
model=fpw.ModelEmotionDetection()

In [ ]:
#return singola emotion
model.get_emotion("I am so happy")

In [ ]:
#return lista emotion con probabilita
model.list_prob("I am so happy")